In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("PROJECT").getOrCreate()

In [0]:
import logging
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("POS_Daily_Transaction_File_Processing").getOrCreate()

In [0]:
# Function to check if mountpoint exists

def mount_exists(mount_point):
    mounts = [mount.mountPoint for mount in dbutils.fs.mounts()]
    return mount_point in mounts


# Define the mount point and storage details

mount_point="/mnt/pos_data_processing"
storage_account_name="pharmacydata2024"
container_name="project-data"
sas_token="DECBZEPbRnh8pmQGIHIRyH9QpredTuDf6nlHoQOn8pHqb1YBPzBHDrvo1uKIojLYh226lFaNS62B+AStdZxCCw=="


# Only create the mount if it doesn't already exist

if not mount_exists(mount_point):
    print(f"Mount point {mount_point} does not exist,creating mount.")

    # Create the mount
    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
        mount_point=mount_point,
        extra_configs={"fs.azure.account.key.pharmacydata2024.blob.core.windows.net":sas_token}
    )
    
    print(f"Mount point {mount_point} created successfully.")
else:
    print(f"Mount point {mount_point} already exists. No action taken.")


Mount point /mnt/pos_data_processing already exists. No action taken.


In [0]:
#List down all directories inside mount point

dbutils.fs.ls(mount_point)

[FileInfo(path='dbfs:/mnt/pos_data_processing/daily-transactional-data/', name='daily-transactional-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/pos_data_processing/joblogs/', name='joblogs/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/pos_data_processing/processed-daily-transactional-data/', name='processed-daily-transactional-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/pos_data_processing/project-data/', name='project-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/pos_data_processing/raw-data/', name='raw-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/pos_data_processing/test_output_from_adb/', name='test_output_from_adb/', size=0, modificationTime=0)]

In [0]:
pos_daily_file="/mnt/pos_data_processing/daily-transactional-data/POS_Sales_Data_2024_demo_testing.csv"
pos_daily_file=spark.read.format("csv").option("header","true").option("inferSchema","true").load(pos_daily_file)
display(pos_daily_file)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T0001,S010,Los Angeles,C005,U051,62,Male,P005,Cancer Treatment (Prescription),Metformin,4,115.25,24-06-2024 11:02,461.0,25.84,32.27,467.43
T0002,S008,Philadelphia,C009,U018,29,Female,P036,Cancer Treatment (Prescription),Omeprazole,1,424.88,10-06-2024 05:53,424.88,44.57,29.7416,410.0516
T0003,S006,San Antonio,C003,U097,62,Female,P030,Bone Health (OTC & Prescription),Olopatadine,1,124.91,05-08-2024 20:57,124.91,9.32,8.7437,124.3337
T0004,S004,San Antonio,C003,U081,30,Female,P019,Antibiotics (Prescription),Ciprofloxacin,1,405.18,01-07-2024 17:52,405.18,24.72,28.3626,408.8226
T0005,S007,Philadelphia,C005,U096,57,Male,P012,Bone Health (OTC & Prescription),Emtricitabine,2,211.43,15-07-2024 20:20,422.86,39.32,29.6002,413.1402
T0006,S003,San Jose,C007,U019,46,Female,P014,Eye Health (OTC & Prescription),Aspirin,1,162.45,19-07-2024 00:33,162.45,15.68,11.3715,158.1415
T0007,S002,Houston,C007,U018,42,Female,P010,Eye Health (OTC & Prescription),Metformin,4,153.53,20-07-2024 00:25,614.12,21.12,42.9884,635.9884
T0008,S010,Dallas,C003,U069,42,Male,P034,Pain Relief (OTC & Prescription),Cisplatin,4,338.24,08-06-2024 16:14,1352.96,19.51,94.7072,1428.1572
T0009,S003,Chicago,C001,U014,18,Male,P004,Gastrointestinal (OTC & Prescription),Gabapentin,1,442.56,12-06-2024 14:42,442.56,44.58,30.9792,428.9592
T0010,S005,San Jose,C005,U007,40,Male,P029,Gastrointestinal (OTC & Prescription),Clotrimazole,4,48.51,05-06-2024 12:35,194.04,40.68,13.5828,166.9428


In [0]:
#Filtering Null Rows if any

df_pos_daily_file=pos_daily_file.dropna()
display(df_pos_daily_file)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T0001,S010,Los Angeles,C005,U051,62,Male,P005,Cancer Treatment (Prescription),Metformin,4,115.25,24-06-2024 11:02,461.0,25.84,32.27,467.43
T0002,S008,Philadelphia,C009,U018,29,Female,P036,Cancer Treatment (Prescription),Omeprazole,1,424.88,10-06-2024 05:53,424.88,44.57,29.7416,410.0516
T0003,S006,San Antonio,C003,U097,62,Female,P030,Bone Health (OTC & Prescription),Olopatadine,1,124.91,05-08-2024 20:57,124.91,9.32,8.7437,124.3337
T0004,S004,San Antonio,C003,U081,30,Female,P019,Antibiotics (Prescription),Ciprofloxacin,1,405.18,01-07-2024 17:52,405.18,24.72,28.3626,408.8226
T0005,S007,Philadelphia,C005,U096,57,Male,P012,Bone Health (OTC & Prescription),Emtricitabine,2,211.43,15-07-2024 20:20,422.86,39.32,29.6002,413.1402
T0006,S003,San Jose,C007,U019,46,Female,P014,Eye Health (OTC & Prescription),Aspirin,1,162.45,19-07-2024 00:33,162.45,15.68,11.3715,158.1415
T0007,S002,Houston,C007,U018,42,Female,P010,Eye Health (OTC & Prescription),Metformin,4,153.53,20-07-2024 00:25,614.12,21.12,42.9884,635.9884
T0008,S010,Dallas,C003,U069,42,Male,P034,Pain Relief (OTC & Prescription),Cisplatin,4,338.24,08-06-2024 16:14,1352.96,19.51,94.7072,1428.1572
T0009,S003,Chicago,C001,U014,18,Male,P004,Gastrointestinal (OTC & Prescription),Gabapentin,1,442.56,12-06-2024 14:42,442.56,44.58,30.9792,428.9592
T0010,S005,San Jose,C005,U007,40,Male,P029,Gastrointestinal (OTC & Prescription),Clotrimazole,4,48.51,05-06-2024 12:35,194.04,40.68,13.5828,166.9428


In [0]:
df_pos_daily_file.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- StoreLocation: string (nullable = true)
 |-- CashierID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- CustomerAge: integer (nullable = true)
 |-- CustomerGender: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- DiscountApplied: double (nullable = true)
 |-- TaxAmount: double (nullable = true)
 |-- TotalAmountAfterTax: double (nullable = true)



In [0]:
#Total Sales Calculation by Date

total_sales_date=df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm")) \
    .groupBy("Date") \
    .agg(sum("TotalAmount").alias("TotalSales")) \
    .orderBy("TotalSales")
total_sales_date.show()

+----------+-----------------+
|      Date|       TotalSales|
+----------+-----------------+
|2024-06-04|            85.89|
|2024-08-05|           124.91|
|2024-06-05|           194.04|
|2024-07-12|           238.14|
|2024-07-15|           422.86|
|2024-06-12|           442.56|
|2024-07-23|            487.8|
|2024-07-20|           614.12|
|2024-06-10|           614.52|
|2024-06-24|           745.28|
|2024-07-19|975.6500000000001|
|2024-06-08|          1352.96|
|2024-07-01|          1727.16|
|2024-06-23|          1921.46|
+----------+-----------------+



In [0]:
#Total Sales Calculation by StoreLocation

total_sales_by_store=df_pos_daily_file.groupBy("StoreLocation").agg(sum("TotalAmount").alias("TotalSales")).orderBy("TotalSales",ascending=False)
total_sales_by_store.show()

+-------------+------------------+
|StoreLocation|        TotalSales|
+-------------+------------------+
|      Chicago|2012.6999999999998|
|  Los Angeles|           1782.98|
| Philadelphia|           1660.94|
|       Dallas|           1352.96|
|      Phoenix|           1028.76|
|     San Jose|            726.66|
|      Houston|            614.12|
|  San Antonio|            530.09|
|New York City|            238.14|
+-------------+------------------+



In [0]:
#Customer Insight

avg_age_customers=df_pos_daily_file.agg({"CustomerAge":"avg"}).alias("AverageCustomerAge")
avg_age_customers.show()

+-----------------+
| avg(CustomerAge)|
+-----------------+
|43.68421052631579|
+-----------------+



In [0]:
#Top Highest Selling Products
top_selling_products=df_pos_daily_file.groupBy("ProductName").agg(sum("TotalAmount").alias("TotalSales")).orderBy("TotalSales",ascending=False)
top_selling_products.show()

+-------------+----------+
|  ProductName|TotalSales|
+-------------+----------+
|    Vitamin D|   1606.26|
|    Cisplatin|    1542.6|
|   Loperamide|   1082.34|
|    Metformin|   1075.12|
|  Olopatadine|    938.11|
|Ciprofloxacin|    892.98|
|Acetaminophen|    839.12|
|   Gabapentin|    442.56|
|   Omeprazole|    424.88|
|Emtricitabine|    422.86|
|   Lisinopril|    238.14|
| Clotrimazole|    194.04|
|      Aspirin|    162.45|
|     Diazepam|     85.89|
+-------------+----------+



In [0]:
#Total Discounts Applied

df_pos_daily_file2=df_pos_daily_file.withColumn("Date",to_timestamp("Timestamp"))
total_discount=df_pos_daily_file2.groupBy("ProductCategory") \
    .agg(sum("DiscountApplied").alias("TotalDiscounts"))\
    .orderBy("TotalDiscounts",ascending=False)
total_discount.show()

+--------------------+------------------+
|     ProductCategory|    TotalDiscounts|
+--------------------+------------------+
|Gastrointestinal ...| 85.25999999999999|
|Eye Health (OTC &...|             74.38|
|Cancer Treatment ...|             70.41|
|Bone Health (OTC ...| 70.10000000000001|
|Antibiotics (Pres...|59.160000000000004|
|AIDS Treatment (P...|             31.21|
|Cardiovascular Di...|             29.24|
|Pain Relief (OTC ...|             19.51|
|Neurological Diso...|              3.62|
|Diabetes (Prescri...|              0.01|
+--------------------+------------------+



In [0]:
#Classify customers based on their total spending into different categories (e.g., low, medium, high)

customer_spending=df_pos_daily_file.groupBy("CustomerID") \
    .agg(sum("TotalAmount").alias("TotalSpent"))

customer_segments=customer_spending.withColumn("SpendingCategory",
    when(col("TotalSpent") < 500, "Low")
    .when((col("TotalSpent") >= 500) & (col("TotalSpent") < 1000), "Medium")
    .otherwise("High"))

customer_segments_sorted=customer_segments.orderBy("TotalSpent")
customer_segments_sorted.show()

+----------+----------+----------------+
|CustomerID|TotalSpent|SpendingCategory|
+----------+----------+----------------+
|      U021|     85.89|             Low|
|      U097|    124.91|             Low|
|      U019|    162.45|             Low|
|      U039|    189.64|             Low|
|      U007|    194.04|             Low|
|      U037|    238.14|             Low|
|      U059|    284.28|             Low|
|      U081|    405.18|             Low|
|      U096|    422.86|             Low|
|      U014|    442.56|             Low|
|      U051|     461.0|             Low|
|      U009|     487.8|             Low|
|      U065|     813.2|          Medium|
|      U057|    839.12|          Medium|
|      U018|    1039.0|            High|
|      U053|   1082.34|            High|
|      U091|   1321.98|            High|
|      U069|   1352.96|            High|
+----------+----------+----------------+



In [0]:
#Monthly Sales Trends

from pyspark.sql import functions as F

df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

df_monthly_sales = df_with_date.withColumn("Year", year(col("Date"))) \
    .withColumn("Month", month(col("Date"))) \
    .groupBy("Year", "Month") \
    .agg(sum("TotalAmount").alias("MonthlySales")) \
    .orderBy("Year", "Month")

display(df_monthly_sales)

Year,Month,MonthlySales
2024,6,5356.71
2024,7,4465.7300000000005
2024,8,124.91


In [0]:
#Calculate Retention Rate

from pyspark.sql import functions as F
from pyspark.sql import Window

# Step 1: Convert Timestamp to Date
df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Calculate Total Purchases by Customer and Group by City
customer_city_purchases = df_with_date.groupBy("CustomerID", "StoreLocation") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

# Step 3: Identify Retained Customers by City
retained_customers_city = customer_city_purchases.groupBy("StoreLocation") \
    .agg(F.count(F.when(col("TotalPurchases") > 1, 1)).alias("RetainedCustomers"),
         F.count("CustomerID").alias("TotalCustomers")) \
    .withColumn("RetentionRateCity", (col("RetainedCustomers") / col("TotalCustomers")) * 100)

# Step 4: Calculate Total Purchases by Customer and Group by Product Category
customer_category_purchases = df_with_date.groupBy("CustomerID", "ProductCategory") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

# Step 5: Identify Retained Customers by Product Category
retained_customers_category = customer_category_purchases.groupBy("ProductCategory") \
    .agg(F.count(F.when(col("TotalPurchases") > 1, 1)).alias("RetainedCustomers"),
         F.count("CustomerID").alias("TotalCustomers")) \
    .withColumn("RetentionRateCategory", (col("RetainedCustomers") / col("TotalCustomers")) * 100)

retained_customers_city.show()
retained_customers_category.show()

+-------------+-----------------+--------------+-----------------+
|StoreLocation|RetainedCustomers|TotalCustomers|RetentionRateCity|
+-------------+-----------------+--------------+-----------------+
|      Phoenix|                0|             2|              0.0|
|       Dallas|                0|             1|              0.0|
|  San Antonio|                0|             2|              0.0|
| Philadelphia|                0|             3|              0.0|
|  Los Angeles|                0|             2|              0.0|
|      Chicago|                0|             3|              0.0|
|     San Jose|                0|             4|              0.0|
|New York City|                0|             1|              0.0|
|      Houston|                0|             1|              0.0|
+-------------+-----------------+--------------+-----------------+

+--------------------+-----------------+--------------+---------------------+
|     ProductCategory|RetainedCustomers|TotalCusto

In [0]:
#Monthly sales and growth rate for each product category in each city

from pyspark.sql import functions as F
from pyspark.sql import Window

# Step 1: Convert Timestamp to Date
df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Create Year and Month Columns
df_with_date = df_with_date.withColumn("Year", year(col("Date"))) \
    .withColumn("Month", month(col("Date")))

# Step 3: Calculate Monthly Sales per Product per City
monthly_sales = df_with_date.groupBy("Year", "Month", "StoreLocation", "ProductCategory") \
    .agg(sum("TotalAmount").alias("MonthlySales")) \
    .orderBy("Year", "Month", "StoreLocation", "ProductCategory")

# Step 4: Calculate Previous Month Sales for Growth Calculation
window_spec = Window.partitionBy("StoreLocation", "ProductCategory").orderBy("Year", "Month")
sales_growth = monthly_sales.withColumn("PrevMonthSales", lag("MonthlySales", 1).over(window_spec)) \
    .withColumn("GrowthRateinPercent",(col("MonthlySales")-col("PrevMonthSales"))/col("PrevMonthSales")*100)

result=sales_growth.select("Year","Month","StoreLocation","ProductCategory","MonthlySales", "PrevMonthSales","GrowthRateinPercent")

result.show()

+----+-----+-------------+--------------------+------------+--------------+-------------------+
|Year|Month|StoreLocation|     ProductCategory|MonthlySales|PrevMonthSales|GrowthRateinPercent|
+----+-----+-------------+--------------------+------------+--------------+-------------------+
|2024|    7|      Chicago|Antibiotics (Pres...|       487.8|          NULL|               NULL|
|2024|    6|      Chicago|Gastrointestinal ...|      442.56|          NULL|               NULL|
|2024|    6|      Chicago|Neurological Diso...|     1082.34|          NULL|               NULL|
|2024|    6|       Dallas|Pain Relief (OTC ...|     1352.96|          NULL|               NULL|
|2024|    7|      Houston|Eye Health (OTC &...|      614.12|          NULL|               NULL|
|2024|    7|  Los Angeles|Antibiotics (Pres...|     1321.98|          NULL|               NULL|
|2024|    6|  Los Angeles|Cancer Treatment ...|       461.0|          NULL|               NULL|
|2024|    7|New York City|Cardiovascular

In [0]:
#Weekly sales and growth rate for each product category in each city

from pyspark.sql import functions as F
from pyspark.sql import Window

# Step 1: Convert Timestamp to Date
df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Create Year and Week Number Columns
df_with_date = df_with_date.withColumn("Year", year(col("Date"))) \
    .withColumn("Week", F.weekofyear(col("Date")))

# Step 3: Calculate Weekly Sales per Product per City
weekly_sales = df_with_date.groupBy("Year", "Week", "StoreLocation", "ProductCategory") \
    .agg(sum("TotalAmount").alias("WeeklySales")) \
    .orderBy("Year", "Week", "StoreLocation", "ProductCategory")

# Step 4: Calculate Previous Week Sales for Growth Calculation
window_spec = Window.partitionBy("StoreLocation", "ProductCategory").orderBy("Year", "Week")
sales_growth = weekly_sales.withColumn("PrevWeekSales", lag("WeeklySales", 1).over(window_spec)) \
    .withColumn("GrowthRateinPercent",(col("WeeklySales")-col("PrevWeekSales"))/col("PrevWeekSales")*100)\
    .orderBy("Year","Week","StoreLocation","ProductCategory")

# Step 5: Select Relevant Columns
result=sales_growth.select("Year","Week","StoreLocation","ProductCategory","WeeklySales","PrevWeekSales", "GrowthRateinPercent")

result.show()


+----+----+-------------+--------------------+-----------+-------------+-------------------+
|Year|Week|StoreLocation|     ProductCategory|WeeklySales|PrevWeekSales|GrowthRateinPercent|
+----+----+-------------+--------------------+-----------+-------------+-------------------+
|2024|  23|       Dallas|Pain Relief (OTC ...|    1352.96|         NULL|               NULL|
|2024|  23|     San Jose|AIDS Treatment (P...|      85.89|         NULL|               NULL|
|2024|  23|     San Jose|Gastrointestinal ...|     194.04|         NULL|               NULL|
|2024|  24|      Chicago|Gastrointestinal ...|     442.56|         NULL|               NULL|
|2024|  24| Philadelphia|Cancer Treatment ...|     424.88|         NULL|               NULL|
|2024|  24|      Phoenix|Bone Health (OTC ...|     189.64|         NULL|               NULL|
|2024|  25|      Chicago|Neurological Diso...|    1082.34|         NULL|               NULL|
|2024|  25|      Phoenix|Bone Health (OTC ...|     839.12|       189.6

Citywise KPI

In [0]:
#Citywise KPI Calculation

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window



# Step 1: Convert Timestamp to Date
df_with_date = df_pos_daily_file.withColumn("Date", to_date(col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Calculate Total Sales and Number of Transactions by City
citywise_sales = df_with_date.groupBy("StoreLocation") \
    .agg(
        F.sum("TotalAmount").alias("TotalSales"),
        F.count("TransactionID").alias("TotalTransactions"),
        F.countDistinct("CustomerID").alias("UniqueCustomers")
    )

# Step 3: Calculate Average Transaction Value
citywise_sales = citywise_sales.withColumn("AvgTransactionValue", col("TotalSales") / col("TotalTransactions"))

# Step 4: Calculate Retained Customers
customer_city_purchases = df_with_date.groupBy("CustomerID", "StoreLocation") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

retained_customers_city = customer_city_purchases.groupBy("StoreLocation") \
    .agg(F.count(F.when(col("TotalPurchases") > 1, 1)).alias("RetainedCustomers"))

# Step 5: Join the retained customers with citywise sales
citywise_kpi = citywise_sales.join(retained_customers_city, "StoreLocation", "left") \
    .fillna(0)

# Step 6: Calculate Retention Rate
citywise_kpi = citywise_kpi.withColumn("RetentionRate", (col("RetainedCustomers") / col("UniqueCustomers")) * 100)

# Step 7: Find Top Selling Product Category by City
top_product_category = df_with_date.groupBy("StoreLocation", "ProductCategory") \
    .agg(F.sum("TotalAmount").alias("CategorySales")) \
    .withColumn("Rank", F.row_number().over(Window.partitionBy("StoreLocation").orderBy(col("CategorySales").desc()))) \
    .filter(col("Rank") == 1) \
    .select("StoreLocation", "ProductCategory", "CategorySales")

# Step 8: Join Top Selling Product Category with Citywise KPI
df_final_city_kpi=citywise_kpi.join(top_product_category, "StoreLocation", "left") \
    .withColumnRenamed("ProductCategory", "TopSellingProductCategory") \
    .withColumnRenamed("CategorySales", "TopSellingCategorySales")\
    .orderBy("CategorySales",ascending=True)

# Show final KPI results
df_final_city_kpi.select("StoreLocation","TotalSales","TotalTransactions","AvgTransactionValue", 
                 "UniqueCustomers","RetainedCustomers","RetentionRate", 
                 "TopSellingProductCategory","TopSellingCategorySales")

display(df_final_city_kpi)

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
New York City,238.14,1,1,238.14,0,0.0,Cardiovascular Diseases (Prescription),238.14
San Antonio,530.09,2,2,265.045,0,0.0,Antibiotics (Prescription),405.18
San Jose,726.66,4,4,181.665,0,0.0,Eye Health (OTC & Prescription),446.72999999999996
Houston,614.12,1,1,614.12,0,0.0,Eye Health (OTC & Prescription),614.12
Philadelphia,1660.94,3,3,553.6466666666666,0,0.0,Diabetes (Prescription & OTC),813.2
Phoenix,1028.76,2,2,514.38,0,0.0,Bone Health (OTC & Prescription),1028.76
Chicago,2012.6999999999998,3,3,670.9,0,0.0,Neurological Disorders (Prescription),1082.34
Los Angeles,1782.98,2,2,891.49,0,0.0,Antibiotics (Prescription),1321.98
Dallas,1352.96,1,1,1352.96,0,0.0,Pain Relief (OTC & Prescription),1352.96


Product Category KPI

In [0]:
#Product Category KPI Calculation

df_with_date = df_pos_daily_file.withColumn("Date", F.to_date(F.col("Timestamp"), "dd-MM-yyyy HH:mm"))

# Step 2: Calculate Total Sales and Number of Transactions by Product Category
categorywise_sales = df_with_date.groupBy("ProductCategory") \
    .agg(
        F.sum("TotalAmount").alias("TotalSales"),
        F.count("TransactionID").alias("TotalTransactions"),
        F.countDistinct("CustomerID").alias("UniqueCustomers")
    )

# Step 3: Calculate Average Transaction Value
categorywise_sales = categorywise_sales.withColumn("AvgTransactionValue", 
                                                   col("TotalSales") / col("TotalTransactions"))

# Step 4: Calculate Retained Customers for Each Product Category
customer_category_purchases = df_with_date.groupBy("CustomerID", "ProductCategory") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

retained_customers_category = customer_category_purchases.groupBy("ProductCategory") \
    .agg(F.count(F.when(col("TotalPurchases") > 1, 1)).alias("RetainedCustomers"))

# Step 5: Join the retained customers with category-wise sales
categorywise_kpi = categorywise_sales.join(retained_customers_category, "ProductCategory", "left") \
    .fillna(0)

# Step 6: Calculate Retention Rate
categorywise_kpi = categorywise_kpi.withColumn("RetentionRate", 
                                               (col("RetainedCustomers") / col("UniqueCustomers")) * 100)

# Step 7: Find Top Selling Product by Category
top_product = df_with_date.groupBy("ProductCategory", "ProductID") \
    .agg(F.sum("TotalAmount").alias("ProductSales")) \
    .withColumn("Rank", F.row_number().over(Window.partitionBy("ProductCategory").orderBy(col("ProductSales").desc()))) \
    .filter(col("Rank") == 1) \
    .select("ProductCategory", "ProductID", "ProductSales")

# Step 8: Join Top Selling Product with Categorywise KPI
df_final_category_kpi = categorywise_kpi.join(top_product, "ProductCategory", "left") \
    .withColumnRenamed("ProductID", "TopSellingProductID") \
    .withColumnRenamed("ProductSales", "TopSellingProductSales")\
    .orderBy("TotalSales",ascending=True)

# Show final KPI results
df_final_category_kpi.select("ProductCategory", "TotalSales", "TotalTransactions", 
                          "AvgTransactionValue", "UniqueCustomers", "RetainedCustomers", 
                          "RetentionRate", "TopSellingProductID", "TopSellingProductSales")


display(df_final_category_kpi)

ProductCategory,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductID,TopSellingProductSales
AIDS Treatment (Prescription),85.89,1,1,85.89,0,0.0,P014,85.89
Cardiovascular Diseases (Prescription),238.14,1,1,238.14,0,0.0,P041,238.14
Gastrointestinal (OTC & Prescription),636.6,2,2,318.3,0,0.0,P004,442.56
Diabetes (Prescription & OTC),813.2,1,1,813.2,0,0.0,P008,813.2
Cancer Treatment (Prescription),885.88,2,2,442.94,0,0.0,P005,461.0
Eye Health (OTC & Prescription),1060.85,3,3,353.6166666666666,0,0.0,P010,614.12
Neurological Disorders (Prescription),1082.34,1,1,1082.34,0,0.0,P002,1082.34
Pain Relief (OTC & Prescription),1352.96,1,1,1352.96,0,0.0,P034,1352.96
Bone Health (OTC & Prescription),1576.5300000000002,4,4,394.13250000000005,0,0.0,P029,839.12
Antibiotics (Prescription),2214.96,3,3,738.32,0,0.0,P032,1321.98


Customer Insights KPI

In [0]:
#Customer Insights KPI Calculation

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

# Step 1: Convert Timestamp to Date and Extract Month
df_with_date = df_pos_daily_file.withColumn("Date", F.to_date(F.col("Timestamp"), "dd-MM-yyyy HH:mm")) \
                                .withColumn("Month", F.date_format(F.col("Date"), "MMM-yyyy"))  # Extract month in 'MMM-yyyy' format

# Step 2: Calculate Total Spending and Number of Transactions by Customer
customer_kpi = df_with_date.groupBy("CustomerID") \
    .agg(
        F.sum("TotalAmount").alias("TotalSpending"),
        F.count("TransactionID").alias("TotalTransactions"),
        F.countDistinct("StoreLocation").alias("UniqueStores"),
    )

# Step 3: Calculate Average Spending Per Transaction
customer_kpi = customer_kpi.withColumn("AvgSpendingPerTransaction", 
                                        F.col("TotalSpending") / F.col("TotalTransactions"))

# Step 4: Calculate Retained Customers
customer_purchases = df_with_date.groupBy("CustomerID") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

retained_customers = customer_purchases.groupBy("CustomerID") \
    .agg(F.count(F.when(F.col("TotalPurchases") > 1, 1)).alias("RetainedStatus"))

# Step 5: Join Retained Status with Customer KPI
customer_kpi = customer_kpi.join(retained_customers, "CustomerID", "left") \
    .fillna(0)

# Step 6: Calculate Retention Rate
customer_kpi = customer_kpi.withColumn("RetentionRate", 
                                        (F.col("RetainedStatus") / F.col("TotalTransactions")) * 100)

# Step 7: Identify Monthly Spending with Month Mapping
# Create month-year columns based on the extracted 'Month' from timestamp
monthly_spending = df_with_date.groupBy("CustomerID", "Month") \
    .agg(F.sum("TotalAmount").alias("MonthlySpending"))

# Step 8: Pivot Monthly Spending to Create Separate Columns with Custom Month Names
# Define month names based on the extracted 'Month'
monthly_spending_pivot = monthly_spending.groupBy("CustomerID") \
    .pivot("Month", ["Jan-2024", "Feb-2024", "Mar-2024", "Apr-2024", "May-2024", "Jun-2024", 
                     "Jul-2024", "Aug-2024", "Sep-2024", "Oct-2024", "Nov-2024", "Dec-2024"]) \
    .agg(F.sum("MonthlySpending")) \
    .withColumnRenamed("Jan-2024", "Jan2024") \
    .withColumnRenamed("Feb-2024", "Feb2024") \
    .withColumnRenamed("Mar-2024", "Mar2024") \
    .withColumnRenamed("Apr-2024", "Apr2024") \
    .withColumnRenamed("May-2024", "May2024") \
    .withColumnRenamed("Jun-2024", "Jun2024") \
    .withColumnRenamed("Jul-2024", "Jul2024") \
    .withColumnRenamed("Aug-2024", "Aug2024") \
    .withColumnRenamed("Sep-2024", "Sep2024") \
    .withColumnRenamed("Oct-2024", "Oct2024") \
    .withColumnRenamed("Nov-2024", "Nov2024") \
    .withColumnRenamed("Dec-2024", "Dec2024")

# Step 9: Join Pivoted Monthly Spending with Customer KPI
final_customer_kpi = customer_kpi.join(monthly_spending_pivot, "CustomerID", "left")

# Step 10: Identify Top Store for Each Customer Based on Spending
top_store = df_with_date.groupBy("CustomerID", "StoreLocation") \
    .agg(F.sum("TotalAmount").alias("StoreSpending")) \
    .withColumn("Rank", F.row_number().over(Window.partitionBy("CustomerID").orderBy(F.col("StoreSpending").desc()))) \
    .filter(F.col("Rank") == 1) \
    .select("CustomerID", "StoreLocation", "StoreSpending")

# Step 11: Final Join with Top Store
df_final_customer_kpi_test = final_customer_kpi.join(top_store, "CustomerID", "left") \
    .withColumnRenamed("StoreLocation", "TopStoreLocation") \
    .withColumnRenamed("StoreSpending", "TopStoreSpending") \
    .orderBy("TotalSpending", ascending=True)

# Show final KPI results with monthly sales columns like Jan2024, Feb2024, etc.
df_final_customer_kpi_test.select("CustomerID", "TotalSpending", "TotalTransactions", 
                          "AvgSpendingPerTransaction", "UniqueStores", 
                          "RetainedStatus", "RetentionRate", 
                          "Jan2024", "Feb2024", "Mar2024", "Apr2024", 
                          "May2024", "Jun2024", "Jul2024", "Aug2024", 
                          "Sep2024", "Oct2024", "Nov2024", "Dec2024", 
                          "TopStoreLocation", "TopStoreSpending")


display(df_final_customer_kpi_test)


CustomerID,TotalSpending,TotalTransactions,UniqueStores,AvgSpendingPerTransaction,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending
U021,85.89,1,1,85.89,0,0.0,null,null,null,null,null,85.89,null,null,null,null,null,null,San Jose,85.89
U097,124.91,1,1,124.91,0,0.0,null,null,null,null,null,null,null,124.91,null,null,null,null,San Antonio,124.91
U019,162.45,1,1,162.45,0,0.0,null,null,null,null,null,null,162.45,null,null,null,null,null,San Jose,162.45
U039,189.64,1,1,189.64,0,0.0,null,null,null,null,null,189.64,null,null,null,null,null,null,Phoenix,189.64
U007,194.04,1,1,194.04,0,0.0,null,null,null,null,null,194.04,null,null,null,null,null,null,San Jose,194.04
U037,238.14,1,1,238.14,0,0.0,null,null,null,null,null,null,238.14,null,null,null,null,null,New York City,238.14
U059,284.28,1,1,284.28,0,0.0,null,null,null,null,null,284.28,null,null,null,null,null,null,San Jose,284.28
U081,405.18,1,1,405.18,0,0.0,null,null,null,null,null,null,405.18,null,null,null,null,null,San Antonio,405.18
U096,422.86,1,1,422.86,0,0.0,null,null,null,null,null,null,422.86,null,null,null,null,null,Philadelphia,422.86
U014,442.56,1,1,442.56,0,0.0,null,null,null,null,null,442.56,null,null,null,null,null,null,Chicago,442.56


In [0]:
#Previous Record Counts

cnt_cust_kpi=spark.sql("SELECT count(*) as final_customer_kpi_prev_count FROM POS_Sales.final_customer_kpi")
cnt_cust_kpi.show()
cnt_category_kpi=spark.sql("SELECT count(*) as final_category_kpi_prev_count FROM POS_Sales.final_category_kpi")
cnt_category_kpi.show()
cnt_city_kpi=spark.sql("SELECT count(*) as final_city_kpi_prev_count FROM POS_Sales.final_city_kpi")
cnt_city_kpi.show()

+-----------------------------+
|final_customer_kpi_prev_count|
+-----------------------------+
|                           18|
+-----------------------------+

+-----------------------------+
|final_category_kpi_prev_count|
+-----------------------------+
|                           10|
+-----------------------------+

+-------------------------+
|final_city_kpi_prev_count|
+-------------------------+
|                        9|
+-------------------------+



Upsert Operations

In [0]:
target_delta_table=DeltaTable.forName(spark,"project_adb.postest.final_customer_kpi_test2")

target_delta_table.alias("target").merge(
    source=df_final_customer_kpi_test3.alias("source"),
    condition="target.CustomerID = source.CustomerID"
).whenMatchedUpdate(
    set={
        "TotalSpending": "target.TotalSpending + source.TotalSpending",
        "TotalTransactions": "target.TotalTransactions + source.TotalTransactions",
        "AvgSpendingPerTransaction": "(target.TotalSpending + source.TotalSpending) / (target.TotalTransactions + source.TotalTransactions)",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
        "Jan2024": "COALESCE(target.Jan2024, 0) + COALESCE(source.Jan2024, 0)",
        "Feb2024": "COALESCE(target.Feb2024, 0) + COALESCE(source.Feb2024, 0)",
        "Mar2024": "COALESCE(target.Mar2024, 0) + COALESCE(source.Mar2024, 0)",
        "Apr2024": "COALESCE(target.Apr2024, 0) + COALESCE(source.Apr2024, 0)",
        "May2024": "COALESCE(target.May2024, 0) + COALESCE(source.May2024, 0)",
        "Jun2024": "COALESCE(target.Jun2024, 0) + COALESCE(source.Jun2024, 0)",
        "Jul2024": "COALESCE(target.Jul2024, 0) + COALESCE(source.Jul2024, 0)",
        "Aug2024": "COALESCE(target.Aug2024, 0) + COALESCE(source.Aug2024, 0)",
        "Sep2024": "COALESCE(target.Sep2024, 0) + COALESCE(source.Sep2024, 0)",
        "Oct2024": "COALESCE(target.Oct2024, 0) + COALESCE(source.Oct2024, 0)",
        "Nov2024": "COALESCE(target.Nov2024, 0) + COALESCE(source.Nov2024, 0)",
        "Dec2024": "COALESCE(target.Dec2024, 0) + COALESCE(source.Dec2024, 0)",
    }
).whenNotMatchedInsert(
    values={
        "CustomerID": "source.CustomerID",
        "TotalSpending": "source.TotalSpending",
        "TotalTransactions": "source.TotalTransactions",
        "AvgSpendingPerTransaction": "source.AvgSpendingPerTransaction",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
        "Jan2024": "source.Jan2024",
        "Feb2024": "source.Feb2024",
        "Mar2024": "source.Mar2024",
        "Apr2024": "source.Apr2024",
        "May2024": "source.May2024",
        "Jun2024": "source.Jun2024",
        "Jul2024": "source.Jul2024",
        "Aug2024": "source.Aug2024",
        "Sep2024": "source.Sep2024",
        "Oct2024": "source.Oct2024",
        "Nov2024": "source.Nov2024",
        "Dec2024": "source.Dec2024",
    }
).execute()

In [0]:
prev_cust_data_kpi_df=spark.table("project_adb.POS_Sales.final_customer_kpi")
display(prev_cust_data_kpi_df)

CustomerID,TotalSpending,TotalTransactions,UniqueStores,AvgSpendingPerTransaction,RetainedStatus,RetentionRate,2024-06,2024-07,2024-08,TopStoreLocation,TopStoreSpending
U021,85.89,1,1,85.89,0,0.0,85.89,null,null,San Jose,85.89
U097,124.91,1,1,124.91,0,0.0,null,null,124.91,San Antonio,124.91
U019,162.45,1,1,162.45,0,0.0,null,162.45,null,San Jose,162.45
U039,189.64,1,1,189.64,0,0.0,189.64,null,null,Phoenix,189.64
U007,194.04,1,1,194.04,0,0.0,194.04,null,null,San Jose,194.04
U037,238.14,1,1,238.14,0,0.0,null,238.14,null,New York City,238.14
U059,284.28,1,1,284.28,0,0.0,284.28,null,null,San Jose,284.28
U081,405.18,1,1,405.18,0,0.0,null,405.18,null,San Antonio,405.18
U096,422.86,1,1,422.86,0,0.0,null,422.86,null,Philadelphia,422.86
U014,442.56,1,1,442.56,0,0.0,442.56,null,null,Chicago,442.56


In [0]:
prev_cust_data_kpi_df.alias("target").merge(
    source=df_final_customer_kpi.alias("source"),
    condition="target.CustomerID = source.CustomerID",
).whenMatchedUpdate(
    set={
        "TotalSpending": "target.TotalSpending + source.TotalSpending",
        "TotalTransactions": "target.TotalTransactions + source.TotalTransactions",
        "AvgSpendingPerTransaction": "source.AvgSpendingPerTransaction",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
    }
).whenNotMatchedInsert(
    values={
        "CustomerID": "source.CustomerID",
        "TotalSpending": "source.TotalSpending",
        "TotalTransactions": "source.TotalTransactions",
        "AvgSpendingPerTransaction": "source.AvgSpendingPerTransaction",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
    }
).execute()


In [0]:
prev_category_data_kpi_df=spark.table("project_adb.POS_Sales.final_category_kpi")
display(prev_category_data_kpi_df)

ProductCategory,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductID,TopSellingProductSales
AIDS Treatment (Prescription),85.89,1,1,85.89,0,0.0,P014,85.89
Cardiovascular Diseases (Prescription),238.14,1,1,238.14,0,0.0,P041,238.14
Gastrointestinal (OTC & Prescription),636.6,2,2,318.3,0,0.0,P004,442.56
Diabetes (Prescription & OTC),813.2,1,1,813.2,0,0.0,P008,813.2
Cancer Treatment (Prescription),885.88,2,2,442.94,0,0.0,P005,461.0
Eye Health (OTC & Prescription),1060.85,3,3,353.6166666666666,0,0.0,P010,614.12
Neurological Disorders (Prescription),1082.34,1,1,1082.34,0,0.0,P002,1082.34
Pain Relief (OTC & Prescription),1352.96,1,1,1352.96,0,0.0,P034,1352.96
Bone Health (OTC & Prescription),1576.5300000000002,4,4,394.13250000000005,0,0.0,P029,839.12
Antibiotics (Prescription),2214.96,3,3,738.32,0,0.0,P032,1321.98


In [0]:
prev_city_data_kpi_df=spark.table("project_adb.POS_Sales.final_city_kpi")
display(prev_city_data_kpi_df)

StoreLocation,TotalSales,TotalTransactions,UniqueCustomers,AvgTransactionValue,RetainedCustomers,RetentionRate,TopSellingProductCategory,TopSellingCategorySales
New York City,238.14,1,1,238.14,0,0.0,Cardiovascular Diseases (Prescription),238.14
San Antonio,530.09,2,2,265.045,0,0.0,Antibiotics (Prescription),405.18
San Jose,726.66,4,4,181.665,0,0.0,Eye Health (OTC & Prescription),446.72999999999996
Houston,614.12,1,1,614.12,0,0.0,Eye Health (OTC & Prescription),614.12
Philadelphia,1660.94,3,3,553.6466666666666,0,0.0,Diabetes (Prescription & OTC),813.2
Phoenix,1028.76,2,2,514.38,0,0.0,Bone Health (OTC & Prescription),1028.76
Chicago,2012.6999999999998,3,3,670.9,0,0.0,Neurological Disorders (Prescription),1082.34
Los Angeles,1782.98,2,2,891.49,0,0.0,Antibiotics (Prescription),1321.98
Dallas,1352.96,1,1,1352.96,0,0.0,Pain Relief (OTC & Prescription),1352.96


After Upsert Record Count Checking

In [0]:
#Updated Record Counts after Upsert Operations

cnt_cust_kpi=spark.sql("SELECT count(*) as final_customer_kpi_updated_count FROM POS_Sales.final_customer_kpi")
cnt_cust_kpi.show()
cnt_category_kpi=spark.sql("SELECT count(*) as final_category_kpi_updated_count FROM POS_Sales.final_category_kpi")
cnt_category_kpi.show()
cnt_city_kpi=spark.sql("SELECT count(*) as final_city_kpi_updated_count FROM POS_Sales.final_city_kpi")
cnt_city_kpi.show()

+--------------------------------+
|final_customer_kpi_updated_count|
+--------------------------------+
|                              18|
+--------------------------------+

+--------------------------------+
|final_category_kpi_updated_count|
+--------------------------------+
|                              10|
+--------------------------------+

+----------------------------+
|final_city_kpi_updated_count|
+----------------------------+
|                           9|
+----------------------------+

